In [ ]:
# import glob
import numpy as np
import os
# import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.sim_data_helpers import get_data_from_fof_folder, get_cosmo_parameters

from galaxy_imaging_clean_helpers import create_circle, subhalo_projection


In [ ]:
base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/"
snapN = 5
gas_resolution=64  # resolution of gas hists
stars_resolution = 128  # resolution of stars hists
dm_resolution = 128  # resolution of dm hists
zoom_box_size = 1000  # size of plottet box
ref_line_length = 200  # length of reference line in the plots

gps_to_use = [   0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
                10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                30, 31, 32, 33,     35, 36, 37, 38, 39,
                40, 41, 42, 43, 44,     46, 47,     49]  # number 34, 45, 48 only ran ultil z = 2

n_cols = 2  # number of columns in plot
n_rows = 7  # number of rows in plot

do_sorting = False
parameter_to_sort_for = 1  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam

# sort plots for cosmo parameter
gp_indices = [i for i in range(len(gps_to_use))]
random.shuffle(gp_indices)  # randomize which boxes are shown - remove this line for debugging to have reproducable results
gp_indices = gp_indices[:n_cols*n_rows]  # only use as many as fit in the plot

if do_sorting:
    sort_param = []
    for gp_index in gp_indices:
        gp = gps_to_use[gp_index]
        path = base_path + f"gridpoint{gp}/"

        Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path)
        params = [Omega0, OmegaBaryon, OmegaLambda, HubbleParam]
        sort_param.append((params[parameter_to_sort_for], gp_index))

    _, sorted_indices = zip(*sorted(sort_param))
else:
    sorted_indices = gp_indices

zoom_box_half = zoom_box_size/2

# get the centeral position from biggest halo in gridpoint0
ref_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/gridpoint0/"
subhalo_mass = get_data_from_fof_folder(ref_path, snapN, "Group", "GroupMass")
index_biggest_halo = max(range(len(subhalo_mass)), key=lambda i: subhalo_mass[i])
reference_halo_cms = get_data_from_fof_folder(ref_path, snapN, "Group", "GroupCM")
reference_halo_cms = reference_halo_cms[index_biggest_halo]  # most massive halo in gp 0

# get coords and masses for all boxes
star_hists = []
star_halfmass_radii = []
gas_hists = []
gas_halfmass_radii = []
dm_hists = []
dm_halfmass_radii = []
for gp_counter in sorted_indices:
    print(f"gp {gp_counter} calcs...")
    path_gp = base_path + f"gridpoint{gps_to_use[gp_counter]}/"
    
    cs_temp, ms_temp, rs_temp = subhalo_projection(path_gp, snapN, particle_type="PartType4", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=500, reference_subhalo_cms=reference_halo_cms)
    cg_temp, mg_temp, rg_temp = subhalo_projection(path_gp, snapN, particle_type="PartType0", zomm_in_box_size=zoom_box_half,
                                             look_for_centeral_bh_radius=500, reference_subhalo_cms=reference_halo_cms)
    cdm_temp, mdm_temp, rdm_temp = subhalo_projection(path_gp, snapN, particle_type="PartType1", zomm_in_box_size=zoom_box_half,
                                         look_for_centeral_bh_radius=500, reference_subhalo_cms=reference_halo_cms)
    
    h_stars, xedges_stars, yedges_stars = np.histogram2d(cs_temp[:, 0], cs_temp[:, 1], weights=ms_temp, bins=stars_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    h_gas, xedges_gas, yedges_gas = np.histogram2d(cg_temp[:, 0], cg_temp[:, 1], weights=mg_temp, bins=gas_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])
    
    h_dm, xedges_dm, y_edges_dm = np.histogram2d(cdm_temp[:, 0], cdm_temp[:, 1], weights=mdm_temp, bins=dm_resolution, range=[[-zoom_box_half, zoom_box_half], [-zoom_box_half, zoom_box_half]])

    star_halfmass_radii.append(rs_temp)
    star_hists.append(h_stars)
    gas_halfmass_radii.append(rg_temp)
    gas_hists.append(h_gas)
    dm_halfmass_radii.append(rdm_temp)
    dm_hists.append(h_dm)


hist_index = 0
norms_stars = []
norms_gas = []
norms_dm = []
for index in range(n_cols):
    max_star_norm = max([i.max() for i in star_hists[hist_index:hist_index+n_rows]])
    max_gas_norm = max([i.max() for i in gas_hists[hist_index:hist_index+n_rows]])
    max_dm_norm = max([i.max() for i in dm_hists[hist_index:hist_index+n_rows]])

    norms_stars.append(mpl.colors.LogNorm(vmax=max_star_norm))
    norms_gas.append(mpl.colors.LogNorm(vmax=max_gas_norm))
    norms_dm.append(mpl.colors.LogNorm(vmax=max_dm_norm))


In [ ]:
fig = plt.figure(figsize=(8, 11))

outer = gridspec.GridSpec(1, n_cols, hspace=0.10)  # Abstand zwischen Blöcken

blocks = [gridspec.GridSpecFromSubplotSpec(n_rows, 3, subplot_spec=outer[i], hspace=0, wspace=0) for i in range(n_cols)]
gp_counter = 0
for block_index in range(len(blocks)):
    current_block = blocks[block_index]
    current_gas_norm = norms_gas[block_index]
    current_stars_norm = norms_stars[block_index]
    current_dm_norm = norms_dm[block_index]
    
    for i in range(n_rows):
        ax_gas = fig.add_subplot(current_block[i, 1])
        ax_stars = fig.add_subplot(current_block[i, 0])
        ax_dm = fig.add_subplot(current_block[i, 2])

        path_gp = base_path + f"gridpoint{gps_to_use[sorted_indices[gp_counter]]}/"
        # print(path_gp)
        Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path_gp)

        # calculate the hist
        h_gas = gas_hists[gp_counter]
        h_stars = star_hists[gp_counter]
        h_dm = dm_hists[gp_counter]

        halfmass_rad_gas = gas_halfmass_radii[gp_counter]
        halfmass_rad_stars = star_halfmass_radii[gp_counter]
        halfmass_rad_dm = dm_halfmass_radii[gp_counter]
        
        # gas
        
        img_gas = ax_gas.imshow(h_gas.T, origin='lower', cmap='plasma',
                        extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                            norm=current_gas_norm, interpolation='nearest')
        
        circle_x, circle_y = create_circle(halfmass_rad_gas)
        ax_gas.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

        ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
        ax_gas.plot(ref_line[0], ref_line[1], c="white")
        ax_gas.text((ref_line[0][0]+ref_line[0][1])/2, ref_line[1][0]*0.95, fr"{ref_line_length} kpc", ha='center', va='bottom', fontsize=5, color="white", weight='bold')
        
        ax_gas.text(zoom_box_half*0.9, zoom_box_half*0.9, "Gas",  va='top', ha='right', fontsize=5, color="white", weight='bold')

        if i == n_rows-1:
            # Colorbar
            cbar_gas = fig.colorbar(img_gas, ax=ax_gas, fraction=0.046, pad=0, orientation="horizontal")
            ticks_gas = cbar_gas.get_ticks()
            physical_bin_volume_gas = (zoom_box_size/gas_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
            new_labels_gas = [f"{ticks_gas/physical_bin_volume_gas:.1e}" for ticks_gas in ticks_gas]
            cbar_gas.set_ticks(ticks_gas)
            cbar_gas.set_ticklabels(new_labels_gas)
            cbar_gas.ax.tick_params(labelsize=5)
            cbar_gas.minorticks_on()
            cbar_gas.set_label(r"Density [$M_\odot$/$Mpc^3$]")

        ax_gas.set_facecolor('black')

        ax_gas.get_xaxis().set_visible(False)
        ax_gas.get_yaxis().set_visible(False)

        ax_gas.set_xlim([-zoom_box_half, zoom_box_half])
        ax_gas.set_ylim([-zoom_box_half, zoom_box_half])

        ax_gas.set_title(fr"$\Omega_m$={Omega0:.2f}, $\Omega_b$={OmegaBaryon:.2f}, $\Omega_\Lambda$={OmegaLambda:.2f}, h={HubbleParam:.2f}", fontsize=8)
        

        # stars

        img_stars = ax_stars.imshow(h_stars.T, origin='lower', cmap='magma',
                        extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                            norm=current_stars_norm, interpolation='nearest')
        
        circle_x, circle_y = create_circle(halfmass_rad_stars)
        ax_stars.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

        ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
        ax_stars.plot(ref_line[0], ref_line[1], c="white")
        ax_stars.text((ref_line[0][0]+ref_line[0][1])/2, ref_line[1][0]*0.95, fr"{ref_line_length} kpc", ha='center', va='bottom', fontsize=5, color="white", weight='bold')
        
        ax_stars.text(zoom_box_half*0.9, zoom_box_half*0.9, "Stars",  va='top', ha='right', fontsize=5, color="white", weight='bold')

        if i == n_rows-1:
            # Colorbar
            cbar_stars = plt.colorbar(img_stars, ax=ax_stars, fraction=0.046, pad=0, orientation="horizontal")
            ticks_stars = cbar_stars.get_ticks()
            physical_bin_volume_stars = (zoom_box_size/stars_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
            new_labels_stars = [f"{ticks_stars/physical_bin_volume_stars:.1e}" for ticks_stars in ticks_stars]
            cbar_stars.set_ticks(ticks_stars)
            cbar_stars.set_ticklabels(new_labels_stars)
            cbar_stars.ax.tick_params(labelsize=5)
            cbar_stars.minorticks_on()
            # cbar_stars.set_label(r"Density [$M_\odot$/$Mpc^3$]")

        ax_stars.set_facecolor('black')

        ax_stars.get_xaxis().set_visible(False)
        ax_stars.get_yaxis().set_visible(False)

        ax_stars.set_xlim([-zoom_box_half, zoom_box_half])
        ax_stars.set_ylim([-zoom_box_half, zoom_box_half])

        # dm
        
        img_dm = ax_dm.imshow(h_dm.T, origin='lower', cmap='viridis',
                        extent=[-zoom_box_half, zoom_box_half, -zoom_box_half, zoom_box_half],
                            norm=current_dm_norm, interpolation='nearest')
        
        circle_x, circle_y = create_circle(halfmass_rad_dm)
        ax_dm.plot(circle_x, circle_y, c="grey", alpha=1, linewidth=0.5)

        ref_line = np.array([[-zoom_box_half*0.8, -zoom_box_half*0.8 + ref_line_length], [-zoom_box_half*0.8, -zoom_box_half*0.8]])
        ax_dm.plot(ref_line[0], ref_line[1], c="white")
        ax_dm.text((ref_line[0][0]+ref_line[0][1])/2, ref_line[1][0]*0.95, fr"{ref_line_length} kpc", ha='center', va='bottom', fontsize=5, color="white", weight='bold')
        
        ax_dm.text(zoom_box_half*0.9, zoom_box_half*0.9, "DM",  va='top', ha='right', fontsize=5, color="white", weight='bold')

        if i == n_rows-1:
            # Colorbar
            cbar_dm = fig.colorbar(img_dm, ax=ax_dm, fraction=0.046, pad=0, orientation="horizontal")
            ticks_dm = cbar_dm.get_ticks()
            physical_bin_volume_dm = (zoom_box_size/dm_resolution)**2 * zoom_box_size  # this is a projection, so one axis has the full length of the box
            new_labels_dm = [f"{ticks_dm/physical_bin_volume_gas:.1e}" for ticks_dm in ticks_dm]
            cbar_dm.set_ticks(ticks_dm)
            cbar_dm.set_ticklabels(new_labels_dm)
            cbar_dm.ax.tick_params(labelsize=5)
            cbar_dm.minorticks_on()
            # cbar_dm.set_label(r"Density [$M_\odot$/$Mpc^3$]")

        ax_dm.set_facecolor('black')

        ax_dm.get_xaxis().set_visible(False)
        ax_dm.get_yaxis().set_visible(False)

        ax_dm.set_xlim([-zoom_box_half, zoom_box_half])
        ax_dm.set_ylim([-zoom_box_half, zoom_box_half])

        gp_counter += 1

        
plt.savefig("plots/galaxy_galery_gas_dm_stars.pdf", format="PDF", dpi=500)
plt.show()